# ProspAI - MVP algoritmo de predição

In [37]:
import pandas as pd
import numpy as np
import sklearn.preprocessing

In [38]:
url = '../data/dataset_prospai.xlsx'
df = pd.read_excel(url)

### Análise dos Dados 


In [39]:
df.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem,Tipo de Campanha Mais Eficaz,Canal Preferido
0,Feminino,45-54,Superior Completo,Sudeste,Normal/Meio-termo,Campanhas de Desconto,Email
1,Feminino,35-44,Superior Completo,Nordeste,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
2,Masculino,35-44,Superior Incompleto,Norte,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
3,Outro,45-54,Fundamental Incompleto,Centro-Oeste,Normal/Meio-termo,Campanhas de Referência,SMS
4,Feminino,65+,Superior Completo,Norte,Formal,Campanhas de Retenção,SMS


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Gênero                        3000 non-null   object
 1   Faixa Etária                  3000 non-null   object
 2   Escolaridade                  3000 non-null   object
 3   Localização                   3000 non-null   object
 4   Linguagem                     3000 non-null   object
 5   Tipo de Campanha Mais Eficaz  3000 non-null   object
 6   Canal Preferido               3000 non-null   object
dtypes: object(7)
memory usage: 164.2+ KB


In [41]:
df.nunique()

Gênero                          3
Faixa Etária                    7
Escolaridade                    7
Localização                     5
Linguagem                       3
Tipo de Campanha Mais Eficaz    5
Canal Preferido                 5
dtype: int64

### Limpeza e organização dos dados

In [42]:
df.rename(columns={'Tipo de Campanha Mais Eficaz': 'Campanha Preferida'}, inplace=True)
df.rename(columns={'Linguagem': 'Linguagem Preferida'}, inplace=True)

In [43]:
df_clean = df

In [44]:
df.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem Preferida,Campanha Preferida,Canal Preferido
0,Feminino,45-54,Superior Completo,Sudeste,Normal/Meio-termo,Campanhas de Desconto,Email
1,Feminino,35-44,Superior Completo,Nordeste,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
2,Masculino,35-44,Superior Incompleto,Norte,Normal/Meio-termo,Campanhas de Novos Produtos,SMS
3,Outro,45-54,Fundamental Incompleto,Centro-Oeste,Normal/Meio-termo,Campanhas de Referência,SMS
4,Feminino,65+,Superior Completo,Norte,Formal,Campanhas de Retenção,SMS


In [46]:
maping = {
    'Masculino' : 0,
    'Feminino': 1,
    'Outro': 2
}
df_clean['Gênero'] = df_clean['Gênero'].replace(maping)

maping2 = {
    '0-17' : 0,
    '18-24': 1,
    '25-34': 2,
    '35-44': 3,
    '45-54': 4,
    '55-64': 5,
    '65+': 6
}
df_clean['Faixa Etária'] = df_clean['Faixa Etária'].replace(maping2)

maping = {
    'Fundamental Incompleto' : 0,
    'Fundamental Completo': 1,
    'Médio Incompleto': 2,
    'Médio Completo': 3,
    'Superior Incompleto': 4,
    'Superior Completo': 5,
    'Pós-graduação': 6
}
df_clean['Escolaridade'] = df_clean['Escolaridade'].replace(maping)

maping = {
    'Sul' : 0,
    'Sudeste': 1,
    'Centro-Oeste': 2,
    'Norte': 3,
    'Nordeste': 4
}
df_clean['Localização'] = df_clean['Localização'].replace(maping)

maping = {
    'Informal': 0,
    'Normal/Meio-termo': 1,
    'Formal': 2
}
df_clean['Linguagem Preferida'] = df_clean['Linguagem Preferida'].replace(maping)

maping = {
    'Campanhas de Desconto': 0,
    'Campanhas de Novos Produtos': 1,
    'Campanhas de Referência': 2,
    'Campanhas de Retenção': 3,
    'Programas de Fidelidade': 4
}
df_clean['Campanha Preferida'] = df_clean['Campanha Preferida'].replace(maping)

maping = {
    'Email': 0,
    'WhatsApp': 1,
    'SMS': 2,
    'Redes Sociais': 3,
    'App Notificações': 4
}
df_clean['Canal Preferido'] = df_clean['Canal Preferido'].replace(maping)

In [47]:
df_clean.head()

,Gênero,Faixa Etária,Escolaridade,Localização,Linguagem Preferida,Campanha Preferida,Canal Preferido
0,1,4,5,1,1,0,0
1,1,3,5,4,1,1,2
2,0,3,4,3,1,1,2
3,2,4,0,2,1,2,2
4,1,6,5,3,2,3,2
